In [9]:
#DATA AUGMENTATION

# Importing necessary functions
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import os

def augment_images(image_path, filename):
		# Loading a sample image
		img = load_img(image_path)
		# Converting the input sample image to an array
		x = img_to_array(img)
		# Reshaping the input image
		x = x.reshape((1, ) + x.shape)

		# Generating and saving 5 augmented samples
		# using the above defined parameters.
		i = 0
		for batch in datagen.flow(x, batch_size = 16,
								save_to_dir ='C:/Users/Camille/Downloads/NEW PATH', #Camille
								# save_to_dir ='D:/Program Files/NEW CROPPED LABELED RESIZED PRES PICS', #Ash
								save_prefix =filename+'_'+str(i), save_format ='jpg'):
			i += 1
			if i > 5:
				break

# Initialising the ImageDataGenerator class.
# We will pass in the augmentation parameters in the constructor.
datagen = ImageDataGenerator(
		rotation_range = 5,
		shear_range = 1,
		zoom_range = 0.2,
		rescale=1)

input_path = 'C:/Users/Camille/Downloads/CROPPED LABELED RESIZED PRES PICS' #Camille
# input_path = 'D:\Program Files\CROPPED LABELED RESIZED PRES PICS' #Ash
for filename in os.listdir(input_path):
	if filename.endswith(".jpg"):
		image_path = os.path.join(input_path, filename)
		augment_images(image_path, filename[:-4])

In [11]:
# # Creating Train / Val / Test folders (One time use)
import os
import numpy as np
import shutil
root_dir = 'C:/Users/Camille/Downloads/splitted data/' #Camille
# root_dir = 'D:/Program Files/SPLIT NEW CROPPED LABELED RESIZED PRES PICS/' #Ash

val_ratio = 0
test_ratio = 0.3

os.makedirs(root_dir +'train/')
os.makedirs(root_dir +'test/')

# Creating partitions of the data after shuffeling
src = 'C:/Users/Camille/Downloads/NEW PATH' # Folder to copy images from --Camille
# src = 'D:/Program Files/NEW CROPPED LABELED RESIZED PRES PICS' #Ash


allFileNames = os.listdir(src)
np.random.shuffle(allFileNames)
train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)* (1 - (val_ratio + test_ratio))), 
                                                           int(len(allFileNames)* (1 - test_ratio))])


train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

print('Total images: ', len(allFileNames))
print('Training: ', len(train_FileNames))
#print('Validation: ', len(val_FileNames))
print('Testing: ', len(test_FileNames))

# Copy-pasting images
for name in train_FileNames:
    shutil.copy(name, root_dir +'train/' )

#for name in val_FileNames:
 #   shutil.copy(name, root_dir +'val/' + cls)

for name in test_FileNames:
    shutil.copy(name, root_dir +'test/' )

Total images:  2585
Training:  1809
Testing:  776


In [71]:
#BINARIZATION
import cv2
import numpy as np

def binary(image):
    (thresh, im_bw) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    thresh = 127
    im_bw = cv2.threshold(image, thresh, 255, cv2.THRESH_BINARY)[1]
    return im_bw

In [72]:
#NOISE REMOVAL
import cv2
def noise(bw):
    # den = bm3d.bm3d(bw, sigma_psd=30/255, stage_arg=bm3d.BM3DStages.HARD_THRESHOLDING)
    den = cv2.fastNlMeansDenoising(bw, None, 10, 7, 15)
    return den

In [73]:
#CED
from scipy import signal
def ced(image):
    m1 = np.array([[5, 5, 5],[-3,0,-3],[-3,-3,-3]])
    m8 = np.array([[-3, 5,5],[-3,0,5],[-3,-3,-3]])
    m7 = np.array([[-3,-3,5],[-3,0,5],[-3,-3,5]])
    m6 = np.array([[-3,-3,-3],[-3,0,5],[-3,5,5]])
    m5 = np.array([[-3, -3, -3],[-3,0,-3],[5,5,5]])
    m4 = np.array([[-3, -3, -3],[5,0,-3],[5,5,-3]])
    m3 = np.array([[5, -3, -3],[5,0,-3],[5,-3,-3]])
    m2 = np.array([[5, 5, -3],[5,0,-3],[-3,-3,-3]])
    list_m = [m1,m2,m3,m4,m5,m6,m7,m8]

    list_e = []
    count = 1
    for m in list_m:
        imgk = signal.convolve2d(image, m,boundary='symm')
        list_e.append(np.abs(imgk))
        out = imgk.astype(np.uint8)
        count += 1
    #Seeking maximum
    count
    e = list_e[0]
    for i in range(len(list_e)):
        e = e*(e>=list_e[i]) + list_e[i]*(e<list_e[i])
        
    e[e>255] = 255
    e=e.astype(np.uint8)
    return e

In [74]:
import cv2
def skeleton(image):
    return cv2.ximgproc.thinning(image)

In [75]:
#PREPROCESSING IMAGES ver 1, 6-8min
import time
import cv2
# import pandas as pd
filename = 'C:/Users/Camille/Downloads/splitted data/train/'
filename2 = 'C:/Users/Camille/Downloads/splitted data/test/'
# filename = 'D:/Program Files/SPLIT NEW CROPPED LABELED RESIZED PRES PICS/train/' #Ash
# filename2 = 'D:/Program Files/SPLIT NEW CROPPED LABELED RESIZED PRES PICS/test/' #Ash
def preprocess(train_dir):
    start_time = time.time()
    list_images = []
    for filename in os.listdir(train_dir):
        image = cv2.imread(os.path.join(train_dir, filename),0)
        img2 = binary(image)
        img3 = noise(img2)
        imgg = cv2.blur(img3,(3,3))
        img4 = ced(imgg)
        img5 = skeleton(img4)
        list_images.append(img5)
    print("--- %s seconds --- image" % (time.time() - start_time))
    list_images = np.array(list_images, np.int32)
    return list_images

list_train_images = preprocess(filename)
list_test_images = preprocess(filename2)
print(list_train_images)
print(list_test_images)

# filepath='C:/Users/Camille/Downloads/pagod na ko/'

# image = cv2.imread('C:/Users/Camille/Downloads/splitted data/train/Hydroxyzine_0_8889.jpg',0)

# img = binary(image)
# img2 = noise(img)

# img3 = cv2.blur(img2, ksize=(3,3))

# img4 = ced(img3)
# img5 = skeleton(img4)

# cv2.imwrite(os.path.join(filepath,"1grey.jpg"), image)  
# cv2.imwrite(os.path.join(filepath,"2binary.jpg"), img)  
# cv2.imwrite(os.path.join(filepath,"3noise.jpg"), img2)  
# cv2.imwrite(os.path.join(filepath,"4blur.jpg"), img3)  
# cv2.imwrite(os.path.join(filepath,"5ced.jpg"), img4) 
# cv2.imwrite(os.path.join(filepath,"6skeleton.jpg"), img5) 

--- 345.5207951068878 seconds --- image
--- 144.41565203666687 seconds --- image
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 

In [76]:
# import tensorflow as tf

# from tensorflow.keras import datasets, layers, models
# import matplotlib.pyplot as plt

# (train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# # print(train_images)
# # image = cv2.imread(os.path.join(filepath,"6skeleton.jpg"))
# # image2 = cv2.imread(os.path.join(filepath,"6skeleton.jpg"))
# # list1 = []
# # list1.append(image)
# # list1.append(image2)
# # list2 = np.array(list1, np.int32)
# print(train_images)


In [77]:
#CNN ASH

In [2]:
#CNN CAM


In [79]:
#CNN TRIX

In [80]:
#CNN CARL